In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hcuppy.ccs import CCSEngine
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import auc, make_scorer, roc_curve, plot_roc_curve, plot_precision_recall_curve, classification_report, f1_score, accuracy_score, roc_auc_score, precision_score, recall_score, balanced_accuracy_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from xgboost import XGBClassifier
from sklearn.base import TransformerMixin
import matplotlib as mpl
import seaborn as sns
import shap
import pickle
import random
import torch
import os
shap.initjs()

In [ ]:
def seed_everything(seed=42):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(1024)

In [ ]:
pat_data = pd.read_csv("./../data/pat_data_new.csv")
ccs_data = pd.read_csv("./../data/ccs_data_new.csv")

In [ ]:
#'mmse_3word_repeat', 'mmse_3word_recall'
pat_data.drop(['mmse_3word_repeat', 'mmse_3word_recall', 'Charlson_Comorbidity_Index', 'complication_sum'], axis=1, inplace=True)

In [ ]:
ccs_dict = {}
file = open("./../data/ccs.txt")
for line in file.readlines():
    strs = line.split(" ")
    ccs = 'ccs_' + strs[0]
    desc = " ".join(strs[1:]).strip()
    ccs_dict[ccs] = desc
file.close()

In [ ]:
cols = [ccs_dict[x] for x in ccs_data.columns[1:]]
ccs_data.columns = ['studyid'] + cols

In [ ]:
Services = np.unique(pat_data.Service.dropna())

In [ ]:
def create_data_by_service(basic_data, ccs_data, service_name):
    # step 1, preprocess basic data 
    pdata = basic_data.copy()
    pats = basic_data.loc[basic_data.Service.isin(service_name), "studyid"]
    pdata = pdata.loc[pdata.Service.isin(service_name)]
    # step 1.1, remove unrelevant variables
    # step 1.2, label the categorical variables
    
    print(pdata.columns)
    # step1, extract diagnosis data
    
    diag_data = ccs_data[ccs_data.studyid.isin(pats)]
    print("____", diag_data.shape, '----')
    
    # rank top diag_data
    names = diag_data.columns[1:]
    counts = np.sum(diag_data.values[:, 1:], axis=0)
    
    print("There are %d patients in the selected services"%diag_data.shape[0])
    ccs_codes = names[(np.sum(diag_data.values[:, 1:], axis=0)/diag_data.shape[0] > 0.1)]
    ccs_counts = counts[(np.sum(diag_data.values[:, 1:], axis=0)/diag_data.shape[0] > 0.1)]
    
    ccs_rank = pd.DataFrame({"ccs_codes" : ccs_codes,
                             "ccs_counts" : ccs_counts})
    
    ccs_rank.sort_values("ccs_counts", ascending=False, inplace=True)
    ccs_rank.index = range(ccs_rank.shape[0])
    
    # extract top diagnosis (ccs) data
    top_ccs_codes = ccs_rank.ccs_codes.values.tolist()
    top_ccs_codes.append("studyid")
    top_ccs_codes_dat = diag_data[top_ccs_codes]
    
    #print("The top", len(ccs_codes),  "CCS groups are used in this service, where all of them appearred in at least 10% of the patients.")
    #print("CCS groups used in this service are: ", top_ccs_codes[:-1])
    #print("The rank of the CCS groups are shown in below:")
    #print(ccs_rank)
    
    # step 3 combine them together
    ret_val = pdata.merge(top_ccs_codes_dat, how="left", on="studyid")
    
    return(ret_val)

In [ ]:
myscoring = {'bal_acc': 'balanced_accuracy',
             'roc_auc': 'roc_auc',
             'ave_pre': 'average_precision',
             'sensitivity': 'recall',
             'f1_score' : 'f1',
             'precision' : 'precision',
             'specificity': make_scorer(recall_score,pos_label=0)
             
            }

In [ ]:
params = {
        'clf__learning_rate': [0.01, 0.05, 0.1],  #3
        'clf__max_depth': [3, 5, 7, 10], # 4
        'clf__min_child_weight': [1, 3, 5], # 3
        'clf__subsample': [0.5, 0.7], # 2
        'clf__colsample_bytree': [0.5, 0.7], # 2
        'clf__n_estimators' : [100, 200], # 2
        'clf__objective': ['binary:logistic']
        }

import matplotlib as mpl
mpl.style.use(plt.rcParamsDefault)
#plt.rcParams['axes.facecolor'] = 'black'
def xgboost(data, title):
    y = data["Frailty"]
    X = data.drop(columns = ["Frailty", "Service", "studyid"])
    
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)
    
    ratio = np.round((len(y)-sum(y))/sum(y), 1)
        
    # categorical data
    
    numerical_features = ['age', 'HEMATOCRIT', 'HEMOGLOBIN', 'PLATELET_COUNT']
    
    categorical_impute_features = ['sex', 'Edu_Years']
    
    categorical_encode_features = ['Race', 'Marital_Status', 'ASA_Anest_Record', 'Patient_Type',
                                   'EmployeeStatus']
    
    
    numerical_transformer = Pipeline(
        steps = [("imputer", SimpleImputer(missing_values=np.nan, strategy="median")),
                 ("scaler", MinMaxScaler())])

    categorical_imputer = Pipeline(
        steps = [("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
                 ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan))]
    )
    
    categorical_encoder = OrdinalEncoder()
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, numerical_features),
            ("cat_imputer", categorical_imputer, categorical_impute_features),
            ("cat_encoder", categorical_encoder, categorical_encode_features),
        ], remainder='passthrough'
    )
    
        
    model = XGBClassifier(random_state = 24, scale_pos_weight=ratio)
    
    pipe = Pipeline(
        steps=[("preprocessor", preprocessor),
               ("clf", model)]
    )
    
    clf = GridSearchCV(pipe, params, scoring="roc_auc", n_jobs=-1,
                       cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=24),
                       verbose=1, refit=True)
    
    nested_score = cross_validate(clf, X=X, y=y, cv=outer_cv, return_train_score=True, 
                                  return_estimator=True, scoring=myscoring)

    ############ SHAP VALUES COMPUTATION FOR EACH FOLD ##############
    iter_shap = 0
    preprocessor.fit_transform(X)
    feature_names = preprocessor.transformers_[0][2] + preprocessor.transformers_[1][2] + preprocessor.transformers_[2][2] + X.columns[preprocessor.transformers_[3][2]].tolist()
                
    aucs_ = []
    confusion_matrices_ = []
    tprs = []
    aucs = []
    
    mean_fpr = np.linspace(0, 1, 100)
    fig, ax=plt.subplots(figsize=(8,8))

    for train_index, test_index in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train = pd.DataFrame(X_train, columns = feature_names)
        X_test = pd.DataFrame(X_test, columns=feature_names)
        
        pickle.dump(train_index, open("./../result/result-XGBoost/Splited Index/train_idx_"+str(iter_shap)+".pkl", 'wb'))
        clf_loaded = nested_score['estimator'][iter_shap]
        # WAHT TO DO
        # SAVE THIS MODEL
        pickle.dump(clf_loaded, open("./../result/result-XGBoost/Trained Models/trained_models_"+str(iter_shap)+".pkl", 'wb'))
        
        X_train_transformed = preprocessor.fit_transform(X_train)
        X_test_transformed = preprocessor.transform(X_test)
        
        X_train_transformed = pd.DataFrame(X_train_transformed, columns=preprocessor.transformers_[0][2] + 
                                                preprocessor.transformers_[1][2] + 
                                                preprocessor.transformers_[2][2] + 
                                                X.columns[preprocessor.transformers_[3][2]].tolist())
        
        X_test_transformed = pd.DataFrame(X_test_transformed, columns=preprocessor.transformers_[0][2] + 
                                                preprocessor.transformers_[1][2] + 
                                                preprocessor.transformers_[2][2] + 
                                                X.columns[preprocessor.transformers_[3][2]].tolist())
        X_train_transformed.index = train_index
        X_test_transformed.index = test_index
        
        pickle.dump(X_train_transformed, open("./../result/result-XGBoost/Splited Data/training_set_"+str(iter_shap)+".pkl", 'wb'))
        pickle.dump(X_test_transformed, open("./../result/result-XGBoost/Splited Data/test_set_"+str(iter_shap)+".pkl", 'wb'))
        ########## Calculate more performance metrics #########
        y_pred = clf_loaded.predict(X_test)
        y_pred_score = clf_loaded.predict_proba(X_test)[:, 1]
        
        _fpr, _tpr, _thresholds = roc_curve(y_pred, y_pred_score, pos_label=1)
    
        viz = plot_roc_curve(nested_score['estimator'][iter_shap].best_estimator_, X_test, y_test,
                         name='ROC fold {}'.format(iter_shap+1),
                         alpha=0.3, lw=1, ax=ax)
        interp_tpr=np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
    
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
        
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_score)
        auc_ = auc(fpr, tpr)
        
        cm = confusion_matrix(y_test, y_pred)
        
        aucs_.append(auc_)
        confusion_matrices_.append(cm)
        ############## Save the predicted result#################
        y_pred_df = pd.Series(y_pred)
        y_pred_df.index = test_index
        pickle.dump(y_pred_df, open("./../result/result-XGBoost/Predict Result/"+"testfold"+str(iter_shap)+'.pkl', 'wb'))
        
        y_pred_score_df = pd.Series(y_pred_score)
        y_pred_score_df.index = test_index
        pickle.dump(y_pred_score_df, open("./../result/result-XGBoost/Predict Probability/"+"testfold"+str(iter_shap)+'.pkl', 'wb'))
        
        # on training data
        train_tmp_df = pd.DataFrame(preprocessor.fit_transform(X_train), columns=feature_names)
        train_explainer = shap.TreeExplainer(clf_loaded.best_estimator_["clf"])
        
        train_shap_values = pd.DataFrame(train_explainer.shap_values(train_tmp_df), columns=feature_names)
        train_shap_values.index = train_index
        
        # on test data
        test_tmp_df = pd.DataFrame(preprocessor.fit(X_train).transform(X_test), columns=feature_names)
        test_explainer = shap.TreeExplainer(clf_loaded.best_estimator_["clf"])
        
        test_shap_values = pd.DataFrame(test_explainer.shap_values(test_tmp_df), columns=feature_names)
        test_shap_values.index = test_index
        
        #print("./../result/SHAP Values/"+"train_"+str(i)+"_fold_"+str(iter_shap)+'.pkl')
        pickle.dump(train_shap_values, open("./../result/result-XGBoost/SHAP Values/"+"trainfold_"+str(iter_shap)+'.pkl', 'wb'))
        pickle.dump(test_shap_values, open("./../result/result-XGBoost/SHAP Values/"+"testfold_"+str(iter_shap)+'.pkl', 'wb'))
        
        iter_shap += 1
        print(iter_shap)
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic curve")
    ax.legend(loc="lower right")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.show()
    return nested_score, aucs_, confusion_matrices_
                    

In [ ]:
nested_score, auc_, confusion_matrix_ =xgboost(create_data_by_service(pat_data, ccs_data, Services), "ALL")

In [ ]:
acc_array = []
for matrix in confusion_matrix_:
    acc = (matrix[0, 0] + matrix[1, 1])/np.sum(matrix)
    acc_array.append(acc)

In [ ]:
np.mean(acc_array)

In [ ]:
np.std(acc_array)

In [ ]:
bal_acc_test_scores = np.round(np.mean(nested_score['test_bal_acc']), 2)
roc_auc_test_scores = np.round(np.mean(nested_score['test_roc_auc']), 2)
ave_pre_test_scores = np.round(np.mean(nested_score['test_ave_pre']), 2)
recall_test_scores = np.round(np.mean(nested_score['test_sensitivity']), 2)
f1_test_scores = np.round(np.mean(nested_score['test_f1_score']), 2)
precision_test_scores = np.round(np.mean(nested_score['test_precision']), 2)
specificity_test_scores = np.round(np.mean(nested_score['test_specificity']), 2)
    
bal_acc_test_std = np.round(np.std(nested_score['test_bal_acc']), 2)
roc_auc_test_std = np.round(np.std(nested_score['test_roc_auc']), 2)
ave_pre_test_std = np.round(np.std(nested_score['test_ave_pre']), 2)
recall_test_std = np.round(np.std(nested_score['test_sensitivity']), 2)
f1_test_std = np.round(np.std(nested_score['test_f1_score']), 2)
precision_test_std = np.round(np.std(nested_score['test_precision']), 2)
specificity_test_std = np.round(np.std(nested_score['test_specificity']), 2)

print('bal_acc: ' + str(bal_acc_test_scores) + ' SD: ' + str(bal_acc_test_std))
print('roc_auc: ' + str(roc_auc_test_scores) + ' SD: ' + str(roc_auc_test_std))
print('ave_pre: ' + str(ave_pre_test_scores)  + ' SD: ' + str(ave_pre_test_std))
print('recall: ' + str(recall_test_scores) + ' SD: ' + str(recall_test_std))
print('f1_score: ' + str(f1_test_scores) + ' SD: ' + str(f1_test_std))
print('precision: ' + str(precision_test_scores) + ' SD: ' + str(precision_test_std))
print('specificity: ' + str(specificity_test_scores) + ' SD: ' + str(specificity_test_std))

In [ ]:
bal_acc_train_scores = np.mean(nested_score['train_bal_acc'])
roc_auc_train_scores = np.mean(nested_score['train_roc_auc'])
ave_pre_train_scores = np.mean(nested_score['train_ave_pre'])
recall_train_scores = np.mean(nested_score['train_sensitivity'])
f1_train_scores = np.mean(nested_score['train_f1_score'])
precision_train_scores = np.mean(nested_score['train_precision'])
specificity_trian_scores = np.mean(nested_score['train_specificity'])

bal_acc_test_scores = np.mean(nested_score['test_bal_acc'])
roc_auc_test_scores = np.mean(nested_score['test_roc_auc'])
ave_pre_test_scores = np.mean(nested_score['test_ave_pre'])
recall_test_scores = np.mean(nested_score['test_sensitivity'])
f1_test_scores = np.mean(nested_score['test_f1_score'])
precision_test_scores = np.mean(nested_score['test_precision'])
specificity_test_scores = np.mean(nested_score['test_specificity'])

print('bal_acc ' + str(bal_acc_test_scores))
print('roc_auc ' + str(roc_auc_test_scores))
print('ave_pre ' + str(ave_pre_test_scores))
print('recall ' + str(recall_test_scores))
print('f1_score ' + str(f1_test_scores))
print('specificity ' + str(specificity_test_scores))
print('precision ' + str(precision_test_scores))
